<a href="https://colab.research.google.com/github/Guy122/Ns3Data/blob/main/LoadAndClean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install mysql-server

In [ ]:
!apt install mysql-server

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl libfcgi-bin libfcgi-perl
  libfcgi0ldbl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2 libprotobuf-lite23
  liburi-perl mecab-ipadic mecab-ipadic-utf8 mecab-utils mysql-client-8.0 mysql-client-core-8.0
  mysql-server-8.0 mysql-server-core-8.0
Suggested packages:
  libdata-dump-perl libipc-sharedcache-perl libbusiness-isbn-perl libwww-perl mailx tinyca
The following NEW packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl libfcgi-bin libfcgi-perl
  libfcgi0ldbl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2 libprotobuf-l

Install python package

In [ ]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 76.6 MB/s eta 0:00:00


Start mysql service

In [ ]:
!service mysql restart

 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


Get into mysql an put the line:

ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root';

for adding root user.

In [ ]:
!mysql

Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 10
Server version: 8.0.39-0ubuntu0.22.04.1 (Ubuntu)

Copyright (c) 2000, 2024, Oracle and/or its affiliates.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

mysql> ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root';
Query OK, 0 rows affected (0.01 sec)

mysql> exit
Bye


Go to sql data dir.

In [ ]:
%cd drive/MyDrive/datans3/

/content/drive/MyDrive/datans3


Adding all sql databases to mysql server. it's will take some time ...

In [ ]:
!mysql -u root -proot < DataNs3_correct.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


Connect to mysql server via python

In [ ]:
# import the connect method
from mysql.connector import connect

# define a connection object
conn = connect(
      user = 'root',
      password = 'root',
      host = 'localhost')

print('A connection object has been created.')

# close the database connection
# conn.close()


A connection object has been created.


Take all databases names and save them into databases_name list.

In [ ]:
mycursor = conn.cursor()

mycursor.execute("SHOW DATABASES;")

myresult = mycursor.fetchall()
databases_name = []
for x in myresult:
  # print(x[0])
  databases_name += [x[0]]

Shows how many databases we have.

In [ ]:
len(databases_name)

44

In [ ]:
x =None
if x > 5 or x == None:
  print(x)

TypeError: '>' not supported between instances of 'NoneType' and 'int'

Drop all faield Databases with simulation steps smaller than 1999 and also where attach_UEs are more the 5, Write there names into wrong_databases list, The correct databases put into correct_databases list.

In [ ]:
correct_databases = []
wrong_databases = []
tables_name = ['UlSINR_Per_RBs','ArrivalRateDl','ArrivalRateUl','Mcs_Dl','Mcs_Ul','RBs_Allocation_Dl','RBs_Allocation_Ul','RSRP','UE_Tx_Power','UlSINR_Per_RBs','harq_Dl','harq_Ul','tbs_Dl','tbs_Ul']
for i in range(len(databases_name)):
  if len(databases_name[i]) > 40:
    mycursor1 = conn.cursor()
    mycursor1.execute(f"use {databases_name[i]};")
    mycursor1.execute(f"SELECT MAX(Attach_UEs) AS max_Attach_UEs FROM Gnbs;")
    myresult1 = mycursor1.fetchall()
    # for x1 in myresult1:
    if myresult1[0][0] == None or myresult1[0][0] > 5:
      mycursor1.execute(f"Drop database {databases_name[i]};")
      wrong_databases += [databases_name[i],]
      # databases_name.remove(databases_name[i])
    else:

      for k in range(len(tables_name)):
        mycursor = conn.cursor()
        mycursor.execute(f"use {databases_name[i]};")
        mycursor.execute(f"SELECT MAX(Simulation_Step) AS max_Simulation_Step FROM {tables_name[k]};")
        myresult = mycursor.fetchall()


        if myresult[0][0] != 1999:
          # if x[0] != 1999:
          #   print(x[0])
          #   correct_databases += [databases_name[i],]
          # else:
          mycursor.execute(f"Drop database {databases_name[i]};")
          wrong_databases += [databases_name[i],]
          # databases_name.remove(databases_name[i])
          break
        if databases_name[i] not in correct_databases:
          correct_databases.append(databases_name[i])


print how many worng databases we had.

In [ ]:
print(len(wrong_databases))

0


print new databases size.

In [ ]:
print(len(databases_name))

44


print how many correct databases we have.

In [ ]:
print(len(correct_databases))

40


Dvide the correct_databases to databases_per_gnb and databases_per_numgnb lists which mean the specific databases per Gnbs and their quantaty.

In [ ]:
# databases_per_numgnb = [[],]*4
databases_per_numgnb3 = []
databases_per_numgnb5 = []
databases_per_numgnb10 = []
databases_per_numgnb20 = []

for i in range(len(correct_databases)):
  gnb_index = ''
  for k in range(len(correct_databases[i])):
    if correct_databases[i][k] =='G':
      # print(correct_databases[i])
      # print(databases_per_numgnb)
      # print(gnb_index)
      if gnb_index == '3':
        databases_per_numgnb3 += [correct_databases[i],]
      if gnb_index == '5':
        databases_per_numgnb5 += [correct_databases[i],]
      if gnb_index == '10':
        databases_per_numgnb10 += [correct_databases[i],]
      if gnb_index == '20':
        databases_per_numgnb20 += [correct_databases[i],]
      break
    gnb_index += correct_databases[i][k]

databases_per_gnb = [databases_per_numgnb3,databases_per_numgnb5,databases_per_numgnb10,databases_per_numgnb20]
databases_per_numgnb =[len(databases_per_numgnb3),len(databases_per_numgnb5),len(databases_per_numgnb10),len(databases_per_numgnb20)]


In [ ]:
databases_per_numgnb

[11, 16, 11, 2]

In [ ]:
!ls

3Gnbs_10Ues_simulation2024x09x29xx03x15x31x084741
3Gnbs_10Ues_simulation2024x09x29xx03x15x31x084741.sql
3Gnbs_10Ues_simulation2024x09x29xx04x35x59x490165
3Gnbs_11Ues_simulation2024x09x29xx02x40x34x086453
3Gnbs_11Ues_simulation2024x09x29xx04x01x46x816362
3Gnbs_13Ues_simulation2024x09x29xx01x12x12x082702
3Gnbs_13Ues_simulation2024x09x29xx01x31x42x997071
3Gnbs_14Ues_simulation2024x09x29xx02x00x16x069922
3Gnbs_7Ues_simulation2024x09x29xx03x38x06x290964
3Gnbs_9Ues_simulation2024x09x29xx03x38x29x982710
3Gnbs_9Ues_simulation2024x09x29xx05x03x44x229224
DataNs3_correct.sql
DataNs3.sql


In [ ]:
!ls

3Gnbs_10Ues_simulation2024x09x29xx03x15x31x084741
3Gnbs_10Ues_simulation2024x09x29xx03x15x31x084741.sql
3Gnbs_10Ues_simulation2024x09x29xx04x35x59x490165
3Gnbs_11Ues_simulation2024x09x29xx02x40x34x086453
3Gnbs_11Ues_simulation2024x09x29xx04x01x46x816362
3Gnbs_13Ues_simulation2024x09x29xx01x12x12x082702
3Gnbs_13Ues_simulation2024x09x29xx01x31x42x997071
3Gnbs_14Ues_simulation2024x09x29xx02x00x16x069922
3Gnbs_7Ues_simulation2024x09x29xx03x38x06x290964
3Gnbs_9Ues_simulation2024x09x29xx03x38x29x982710
3Gnbs_9Ues_simulation2024x09x29xx05x03x44x229224
DataNs3_correct1.sql
DataNs3_correct.sql
DataNs3.sql


In [ ]:
!mysqldump -u root -p --all-databases > DataNs3_correct1.sql

Enter password: 



mysqldump -u root -p --all-databases > DataNs3_correct.sql

---

